In [5]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

df_all_data = pd.read_csv('./dataset/storm_event_details_2010.csv')

for i in range(2011,2021):
    df_temp = pd.read_csv(f'./dataset/storm_event_details_{i}.csv')
    df_all_data = df_all_data.append(df_temp, ignore_index=True)


In [22]:
df_hur = df_all_data[df_all_data['EVENT_TYPE']=='Tornado']
df_hur = df_hur.drop(columns=['TOR_OTHER_WFO', 'BEGIN_YEARMONTH', ''
                                           'TOR_OTHER_CZ_STATE','TOR_OTHER_CZ_FIPS','TOR_OTHER_CZ_NAME','DATA_SOURCE','EPISODE_NARRATIVE',
                                            'EVENT_NARRATIVE','WFO','SOURCE','CZ_TIMEZONE','BEGIN_AZIMUTH','END_AZIMUTH','BEGIN_LAT',
                                            'END_LAT','BEGIN_LON','END_LON','STATE_FIPS','BEGIN_RANGE','END_RANGE','DAMAGE_CROPS',
                                            'BEGIN_TIME','END_TIME','BEGIN_LOCATION','END_LOCATION','FLOOD_CAUSE','MAGNITUDE_TYPE',
                                            'MAGNITUDE','CZ_FIPS','CZ_TYPE','CZ_NAME','CATEGORY'])
cols = ['INJURIES_INDIRECT', 'INJURIES_DIRECT', 'DEATHS_INDIRECT', 'DEATHS_DIRECT']
df_hur = df_hur.assign(HARM_TOTAL=df_hur[cols].sum(1)).drop(cols,1)
df_hur['TOR_AREA'] = df_hur['TOR_LENGTH']*df_hur['TOR_WIDTH']
df_hur = df_hur.drop(columns=['TOR_LENGTH', 'TOR_WIDTH'])

/tmp/ipykernel_2877/645133373.py:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  df_hur = df_hur.assign(HARM_TOTAL=df_hur[cols].sum(1)).drop(cols,1)


In [23]:
df_hur = df_hur.sort_values('HARM_TOTAL', ascending=False)
df_hur.info()
df_hur.head(20)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14988 entries, 100619 to 688592
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BEGIN_YEARMONTH  14988 non-null  int64  
 1   BEGIN_DAY        14988 non-null  int64  
 2   END_YEARMONTH    14988 non-null  int64  
 3   END_DAY          14988 non-null  int64  
 4   EPISODE_ID       14988 non-null  int64  
 5   EVENT_ID         14988 non-null  int64  
 6   STATE            14988 non-null  object 
 7   YEAR             14988 non-null  int64  
 8   MONTH_NAME       14988 non-null  object 
 9   EVENT_TYPE       14988 non-null  object 
 10  BEGIN_DATE_TIME  14988 non-null  object 
 11  END_DATE_TIME    14988 non-null  object 
 12  DAMAGE_PROPERTY  12945 non-null  object 
 13  TOR_F_SCALE      14988 non-null  object 
 14  HARM_TOTAL       14988 non-null  int64  
 15  TOR_AREA         14988 non-null  float64
dtypes: float64(1), int64(8), object(7)
memory usage: 1.9

,BEGIN_YEARMONTH,BEGIN_DAY,END_YEARMONTH,END_DAY,EPISODE_ID,EVENT_ID,STATE,YEAR,MONTH_NAME,EVENT_TYPE,BEGIN_DATE_TIME,END_DATE_TIME,DAMAGE_PROPERTY,TOR_F_SCALE,HARM_TOTAL,TOR_AREA
100619,201105,22,201105,22,49972,296617,MISSOURI,2011,May,Tornado,22-MAY-11 16:40:00,22-MAY-11 17:00:00,2.80B,EF5,1311,14080.00
134758,201104,27,201104,27,50455,314662,ALABAMA,2011,April,Tornado,27-APR-11 15:48:00,27-APR-11 16:35:00,1.50B,EF4,852,73550.40
136287,201104,27,201104,27,50455,314663,ALABAMA,2011,April,Tornado,27-APR-11 16:35:00,27-APR-11 17:14:00,700.00M,EF4,720,84578.00
344213,201512,26,201512,26,101364,606471,TEXAS,2015,December,Tornado,26-DEC-15 18:46:00,26-DEC-15 18:59:00,26.00M,EF4,478,5065.50
265662,201305,20,201305,20,74673,451572,OKLAHOMA,2013,May,Tornado,20-MAY-13 14:04:00,20-MAY-13 14:35:00,2.00B,EF5,231,22800.00
75537,201104,27,201104,27,49915,300457,TENNESSEE,2011,April,Tornado,27-APR-11 19:34:00,27-APR-11 19:56:00,23.00M,EF4,204,16320.00
122118,201106,1,201106,1,52130,323440,MASSACHUSETTS,2011,June,Tornado,01-JUN-11 15:17:00,01-JUN-11 16:09:00,227.60M,EF3,203,27737.60
312392,201404,27,201404,27,83594,504760,ARKANSAS,2014,April,Tornado,27-APR-14 18:25:00,27-APR-14 18:59:00,210.00M,EF4,179,31996.80
632636,202003,3,202003,3,146901,882999,TENNESSEE,2020,March,Tornado,03-MAR-20 00:32:00,03-MAR-20 00:54:00,1.09B,EF3,172,24713.50
599455,201905,27,201905,27,136052,816403,OHIO,2019,May,Tornado,27-MAY-19 21:41:00,27-MAY-19 22:13:00,500.00M,EF4,167,19078.50
